In [2]:
import pandas as pd

# Dosya yolları
base_path = "/kaggle/input/airlines-ml/"

files = [
    "airline_2014_cleaned.txt",
    "airline_2015_cleaned.txt",
    "airline_2016_cleaned.txt",
    "airline_2017_cleaned.txt",
    "airline_2018_cleaned.txt"
]

dfs = []

for f in files:
    print("Loading:", base_path + f)
    df_temp = pd.read_csv(base_path + f)
    dfs.append(df_temp)

# Alt alta birleştir
df = pd.concat(dfs, ignore_index=True)
print("Final shape:", df.shape)

df.head()


Loading: /kaggle/input/airlines-ml/airline_2014_cleaned.txt
Loading: /kaggle/input/airlines-ml/airline_2015_cleaned.txt
Loading: /kaggle/input/airlines-ml/airline_2016_cleaned.txt
Loading: /kaggle/input/airlines-ml/airline_2017_cleaned.txt
Loading: /kaggle/input/airlines-ml/airline_2018_cleaned.txt
Final shape: (30144615, 30)


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_DATETIME,CRS_DEP_HOUR,CRS_DEP_MINUTE
0,2014-01-01,AA,2377,ICT,DFW,1135,1144.0,9.0,12.0,1156.0,...,57.0,328.0,NaN,NaN,NaN,NaN,NaN,2014-01-01 11:35:00,11,35
1,2014-01-01,AA,2378,MIA,TPA,2225,2220.0,-5.0,14.0,2234.0,...,37.0,204.0,NaN,NaN,NaN,NaN,NaN,2014-01-01 22:25:00,22,25
2,2014-01-01,EV,2500,DFW,HOU,2105,NaN,NaN,NaN,NaN,...,NaN,247.0,NaN,NaN,NaN,NaN,NaN,2014-01-01 21:05:00,21,5
3,2014-01-01,EV,2502,CRW,DFW,1655,1805.0,70.0,6.0,1811.0,...,150.0,946.0,0.0,0.0,0.0,0.0,65.0,2014-01-01 16:55:00,16,55
4,2014-01-01,EV,2502,DFW,CRW,1320,1440.0,80.0,9.0,1449.0,...,101.0,946.0,6.0,0.0,0.0,0.0,64.0,2014-01-01 13:20:00,13,20


In [3]:
# 1) Classification target: ARR_DELAY > 15 ?
df["Delayed15"] = (df["ARR_DELAY"] > 15).astype(int)

# 2) Regression target: negatifler 0 olsun
df["ARR_DELAY_POSITIVE"] = df["ARR_DELAY"].apply(lambda x: max(x, 0))


In [4]:
df["FL_DATE"] = pd.to_datetime(df["FL_DATE"])


In [5]:
df["Year"] = df["FL_DATE"].dt.year
df["Month"] = df["FL_DATE"].dt.month
df["DayOfMonth"] = df["FL_DATE"].dt.day
df["DayOfWeek"] = df["FL_DATE"].dt.dayofweek   # 0=Mon, 6=Sun


In [6]:
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"

df["Season"] = df["Month"].apply(get_season)


In [7]:
def hhmm_to_hour(x):
    try:
        x = int(x)
        hour = x // 100
        minute = x % 100
        if hour >= 24:
            hour = 0
        return hour
    except:
        return None

df["CRS_Dep_Hour"] = df["CRS_DEP_TIME"].apply(hhmm_to_hour)


In [8]:
def time_of_day(h):
    if h < 6:
        return "Night"
    elif h < 12:
        return "Morning"
    elif h < 18:
        return "Afternoon"
    else:
        return "Evening"

df["TimeOfDay"] = df["CRS_Dep_Hour"].apply(time_of_day)


In [9]:
df["IsWeekend"] = df["DayOfWeek"].isin([5,6]).astype(int)


In [10]:
df["Route"] = df["ORIGIN"] + "-" + df["DEST"]


In [11]:
def distance_bucket(d):
    if d < 500:
        return "Short"
    elif d < 1500:
        return "Medium"
    else:
        return "Long"

df["DistanceBucket"] = df["DISTANCE"].apply(distance_bucket)


In [14]:
import pandas as pd

# --------------------------
# 1) Sadece ML için gerekli kolonları alıyoruz
# --------------------------
ml_cols = [
    'OP_CARRIER', 'ORIGIN', 'DEST', 'Route',
    'CRS_Dep_Hour', 'DayOfWeek', 'Month', 
    'DistanceBucket', 'Season', 'TimeOfDay', 'IsWeekend',
    'DISTANCE', 'CRS_ELAPSED_TIME',
    'Delayed15',              # classification target
    'ARR_DELAY_POSITIVE'      # regression target
]

df_ml = df[ml_cols].copy()


# --------------------------
# 2) Kategorik kolonları doğru tipe çeviriyoruz
# --------------------------
cat_cols = [
    'OP_CARRIER', 'ORIGIN', 'DEST', 'Route',
    'DistanceBucket', 'Season', 'TimeOfDay'
]

for col in cat_cols:
    df_ml[col] = df_ml[col].astype('category')

# Boolean kolonları int yapıyoruz
df_ml['IsWeekend'] = df_ml['IsWeekend'].astype(int)

print("Final ML dataset shape:", df_ml.shape)
df_ml.head()


Final ML dataset shape: (30144615, 15)


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,OP_CARRIER,ORIGIN,DEST,Route,CRS_Dep_Hour,DayOfWeek,Month,DistanceBucket,Season,TimeOfDay,IsWeekend,DISTANCE,CRS_ELAPSED_TIME,Delayed15,ARR_DELAY_POSITIVE
0,AA,ICT,DFW,ICT-DFW,11,2,1,Short,Winter,Morning,0,328.0,85.0,0,2.0
1,AA,MIA,TPA,MIA-TPA,22,2,1,Short,Winter,Evening,0,204.0,60.0,0,0.0
2,EV,DFW,HOU,DFW-HOU,21,2,1,Short,Winter,Evening,0,247.0,60.0,0,NaN
3,EV,CRW,DFW,CRW-DFW,16,2,1,Medium,Winter,Afternoon,0,946.0,185.0,1,65.0
4,EV,DFW,CRW,DFW-CRW,13,2,1,Medium,Winter,Afternoon,0,946.0,125.0,1,70.0


In [16]:
df_ml.columns

Index(['OP_CARRIER', 'ORIGIN', 'DEST', 'Route', 'CRS_Dep_Hour', 'DayOfWeek',
       'Month', 'DistanceBucket', 'Season', 'TimeOfDay', 'IsWeekend',
       'DISTANCE', 'CRS_ELAPSED_TIME', 'Delayed15', 'ARR_DELAY_POSITIVE'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# 1) Feature set
features_clf = ['OP_CARRIER', 'ORIGIN', 'DEST', 'Route',
                'CRS_Dep_Hour', 'DayOfWeek', 'Month',
                'DistanceBucket', 'Season', 'IsWeekend']

X = df[features_clf]
y = df['Delayed15']

# 2) Categorical / numerical separation
categorical_cols = ['OP_CARRIER', 'ORIGIN', 'DEST', 'Route',
                    'DistanceBucket', 'Season']
numeric_cols = ['CRS_Dep_Hour', 'DayOfWeek', 'Month', 'IsWeekend']

# 3) Preprocessing
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ("num", "passthrough", numeric_cols)
    ]
)

# 4) Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5) Logistic Regression Model
log_clf = Pipeline(steps=[('preprocess', preprocess),
                         ('model', LogisticRegression(max_iter=200))])

log_clf.fit(X_train, y_train)
y_pred_log = log_clf.predict(X_test)
y_proba_log = log_clf.predict_proba(X_test)[:, 1]

print("📌 Logistic Regression")
print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("F1:", f1_score(y_test, y_pred_log))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_log))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_log))

# 6) Random Forest Model
rf_clf = Pipeline(steps=[('preprocess', preprocess),
                        ('model', RandomForestClassifier())])

rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
y_proba_rf = rf_clf.predict_proba(X_test)[:, 1]

print("\n📌 Random Forest")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("F1:", f1_score(y_test, y_pred_rf))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_rf))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_rf))


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1) Feature Set
features_reg = ['OP_CARRIER','ORIGIN','DEST','Route',
                'CRS_Dep_Hour','DayOfWeek','Month',
                'DistanceBucket','Season','IsWeekend',
                'CRS_ELAPSED_TIME','DISTANCE']

X = df[features_reg]
y = df['ARR_DELAY_POSITIVE']

# 2) Preprocessing
categorical_cols = ['OP_CARRIER','ORIGIN','DEST','Route',
                    'DistanceBucket','Season']
numeric_cols = ['CRS_Dep_Hour','DayOfWeek','Month','IsWeekend',
                'CRS_ELAPSED_TIME','DISTANCE']

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ("num", "passthrough", numeric_cols)
    ]
)

# 3) Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4) Linear Regression
reg_model = Pipeline(steps=[('preprocess', preprocess),
                            ('model', LinearRegression())])

reg_model.fit(X_train, y_train)
y_pred = reg_model.predict(X_test)

print("📌 Linear Regression Results")
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))
